# GigE Camera Test Setup

This notebook helps you test and configure your GigE cameras for the USDA vision project.

## Key Features:
- Test camera connectivity
- Display images inline (no GUI needed)
- Save test images/videos to `/storage`
- Configure camera parameters
- Test recording functionality

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime
import time
from pathlib import Path
import imageio
from tqdm import tqdm

# Configure matplotlib for inline display
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['image.cmap'] = 'gray'

print("✅ All imports successful!")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

✅ All imports successful!
OpenCV version: 4.11.0
NumPy version: 2.3.2


## Utility Functions

In [2]:
def display_image(image, title="Image", figsize=(10, 8)):
    """Display image inline in Jupyter notebook"""
    plt.figure(figsize=figsize)
    if len(image.shape) == 3:
        # Convert BGR to RGB for matplotlib
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image_rgb)
    else:
        plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def save_image_to_storage(image, filename_prefix="test_image"):
    """Save image to /storage with timestamp"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{filename_prefix}_{timestamp}.jpg"
    filepath = f"/storage/{filename}"
    
    success = cv2.imwrite(filepath, image)
    if success:
        print(f"✅ Image saved: {filepath}")
        return filepath
    else:
        print(f"❌ Failed to save image: {filepath}")
        return None

def create_storage_subdir(subdir_name):
    """Create subdirectory in /storage"""
    path = Path(f"/storage/{subdir_name}")
    path.mkdir(exist_ok=True)
    print(f"📁 Directory ready: {path}")
    return str(path)

def list_available_cameras():
    """List all available camera devices"""
    print("🔍 Scanning for available cameras...")
    available_cameras = []
    
    # Test camera indices 0-10
    for i in range(11):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            ret, frame = cap.read()
            if ret:
                available_cameras.append(i)
                print(f"📷 Camera {i}: Available (Resolution: {frame.shape[1]}x{frame.shape[0]})")
            cap.release()
        else:
            # Try with different backends for GigE cameras
            cap = cv2.VideoCapture(i, cv2.CAP_GSTREAMER)
            if cap.isOpened():
                ret, frame = cap.read()
                if ret:
                    available_cameras.append(i)
                    print(f"📷 Camera {i}: Available via GStreamer (Resolution: {frame.shape[1]}x{frame.shape[0]})")
                cap.release()
    
    if not available_cameras:
        print("❌ No cameras found")
    
    return available_cameras

print("✅ Utility functions loaded!")

✅ Utility functions loaded!


## Step 1: Check Storage Directory

In [3]:
# Check storage directory
storage_path = Path("/storage")
print(f"Storage directory exists: {storage_path.exists()}")
print(f"Storage directory writable: {os.access('/storage', os.W_OK)}")

# Create test subdirectories
test_images_dir = create_storage_subdir("test_images")
test_videos_dir = create_storage_subdir("test_videos")
camera1_dir = create_storage_subdir("camera1")
camera2_dir = create_storage_subdir("camera2")

Storage directory exists: True
Storage directory writable: True
📁 Directory ready: /storage/test_images
📁 Directory ready: /storage/test_videos
📁 Directory ready: /storage/camera1
📁 Directory ready: /storage/camera2


## Step 2: Scan for Available Cameras

In [4]:
# Scan for cameras
cameras = list_available_cameras()
print(f"\n📊 Summary: Found {len(cameras)} camera(s): {cameras}")

🔍 Scanning for available cameras...
❌ No cameras found

📊 Summary: Found 0 camera(s): []


[ WARN:0@9.977] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ WARN:0@9.977] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=-1, req=-2140645888
[ WARN:0@9.977] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 9
[ WARN:0@9.977] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=-1, req=-2140645888
[ WARN:0@9.977] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 9
[ERROR:0@9.977] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@9.977] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ WARN:0@9.977] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=-1, req=-2140645888
[ WARN:0@9.977] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 9
[ WARN:0@9.977] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=-1, req=-214064588

## Step 3: Test Individual Camera

In [5]:
# Test a specific camera (change camera_id as needed)
camera_id = 0  # Change this to test different cameras

print(f"🔧 Testing camera {camera_id}...")

# Try different backends for GigE cameras
backends_to_try = [
    (cv2.CAP_ANY, "Default"),
    (cv2.CAP_GSTREAMER, "GStreamer"),
    (cv2.CAP_V4L2, "V4L2"),
    (cv2.CAP_FFMPEG, "FFmpeg")
]

successful_backend = None
cap = None

for backend, name in backends_to_try:
    print(f"  Trying {name} backend...")
    cap = cv2.VideoCapture(camera_id, backend)
    if cap.isOpened():
        ret, frame = cap.read()
        if ret:
            print(f"  ✅ {name} backend works!")
            successful_backend = (backend, name)
            break
        else:
            print(f"  ❌ {name} backend opened but can't read frames")
    else:
        print(f"  ❌ {name} backend failed to open")
    cap.release()

if successful_backend:
    backend, backend_name = successful_backend
    cap = cv2.VideoCapture(camera_id, backend)
    
    # Get camera properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"\n📷 Camera {camera_id} Properties ({backend_name}):")
    print(f"  Resolution: {width}x{height}")
    print(f"  FPS: {fps}")
    
    # Capture a test frame
    ret, frame = cap.read()
    if ret:
        print(f"  Frame shape: {frame.shape}")
        print(f"  Frame dtype: {frame.dtype}")
        
        # Display the frame
        display_image(frame, f"Camera {camera_id} Test Frame")
        
        # Save test image
        save_image_to_storage(frame, f"camera_{camera_id}_test")
    else:
        print("  ❌ Failed to capture frame")
    
    cap.release()
else:
    print(f"❌ Camera {camera_id} not accessible with any backend")

🔧 Testing camera 0...
  Trying Default backend...
  ❌ Default backend failed to open
  Trying GStreamer backend...
  ❌ GStreamer backend failed to open
  Trying V4L2 backend...
  ❌ V4L2 backend failed to open
  Trying FFmpeg backend...
  ❌ FFmpeg backend failed to open
❌ Camera 0 not accessible with any backend


[ WARN:0@27.995] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ WARN:0@27.995] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=-1, req=-2140645888
[ WARN:0@27.995] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 9
[ WARN:0@27.995] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=-1, req=-2140645888
[ WARN:0@27.995] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 9
[ERROR:0@27.995] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range
[ WARN:0@27.996] global cap_v4l.cpp:913 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ WARN:0@27.996] global cap.cpp:478 open VIDEOIO(V4L2): backend is generally available but can't be used to capture by index
[ WARN:0@27.996] global cap.cpp:478 open VIDEOIO(FFMPEG): backend is generally available but can't be used to capture by index


## Step 4: Test Video Recording

In [ ]:
# Test video recording
def test_video_recording(camera_id, duration_seconds=5, fps=30):
    """Test video recording from camera"""
    print(f"🎥 Testing video recording from camera {camera_id} for {duration_seconds} seconds...")
    
    # Open camera
    cap = cv2.VideoCapture(camera_id)
    if not cap.isOpened():
        print(f"❌ Cannot open camera {camera_id}")
        return None
    
    # Get camera properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Create video writer
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    video_filename = f"/storage/test_videos/camera_{camera_id}_test_{timestamp}.mp4"
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(video_filename, fourcc, fps, (width, height))
    
    if not out.isOpened():
        print("❌ Cannot create video writer")
        cap.release()
        return None
    
    # Record video
    frames_to_capture = duration_seconds * fps
    frames_captured = 0
    
    print(f"Recording {frames_to_capture} frames...")
    
    with tqdm(total=frames_to_capture, desc="Recording") as pbar:
        start_time = time.time()
        
        while frames_captured < frames_to_capture:
            ret, frame = cap.read()
            if ret:
                out.write(frame)
                frames_captured += 1
                pbar.update(1)
                
                # Display first frame
                if frames_captured == 1:
                    display_image(frame, f"First frame from camera {camera_id}")
            else:
                print(f"❌ Failed to read frame {frames_captured}")
                break
    
    # Cleanup
    cap.release()
    out.release()
    
    elapsed_time = time.time() - start_time
    actual_fps = frames_captured / elapsed_time
    
    print(f"✅ Video saved: {video_filename}")
    print(f"📊 Captured {frames_captured} frames in {elapsed_time:.2f}s")
    print(f"📊 Actual FPS: {actual_fps:.2f}")
    
    return video_filename

# Test recording (change camera_id as needed)
if cameras:  # Only test if cameras were found
    test_camera = cameras[0]  # Use first available camera
    video_file = test_video_recording(test_camera, duration_seconds=3)
else:
    print("⚠️ No cameras available for video test")